In [ ]:
import pandas as pd
import numpy as np
from top2vec import Top2Vec
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
import pickle
import gensim.corpora as corpora
from gensim.models import CoherenceModel

In [ ]:
df = pd.read_csv('data/complete-clean-preprocessed-data-2010-2020.tsv', sep='\t')
# df

In [ ]:
def prep_topic_words(topic_model, df):
    tokens = [list(simple_preprocess(str(s), deacc=True)) for s in df['preprocessed_hlead']]
    dictionary = corpora.Dictionary(tokens)
    corpus = [dictionary.doc2bow(text) for text in tokens]
    topic_words, _, _ = model.get_topics()

    return topic_words, tokens, corpus, dictionary

def evaluation(topic_words, data, dictionary, corpus):
    coherence_model = CoherenceModel(topics= [s.tolist() for s in topic_words], texts=data, dictionary=dictionary, coherence='c_v')
    coherence = coherence_model.get_coherence()
#     perplexity = model.log_perplexity(corpus)
    return coherence#, perplexity

def hyperparameter_tuning(df, name):
    # Change depending on models one wants to check
    umaps = [{'n_neighbors': 5, 'n_components': 5, 'metric': 'cosine', 'low_memory': False, 'random_state': 42},
             {'n_neighbors': 10, 'n_components': 5, 'metric': 'cosine', 'low_memory': False, 'random_state': 42},
             {'n_neighbors': 15, 'n_components': 5, 'metric': 'cosine', 'low_memory': False, 'random_state': 42},
             {'n_neighbors': 5, 'n_components': 10, 'metric': 'cosine', 'low_memory': False, 'random_state': 42},
             {'n_neighbors': 10, 'n_components': 10, 'metric': 'cosine', 'low_memory': False, 'random_state': 42},
             {'n_neighbors': 15, 'n_components': 10, 'metric': 'cosine', 'low_memory': False, 'random_state': 42},
             ]
    coherences = []
    models = []
#     perplexities = []
    num = 0
    for umap in umaps:
        model = Top2Vec(df['preprocessed_hlead'].tolist(), umap_args=umap)
        models.append(model)
        topic_words, tokens, corpus, dictionary = prep_topic_words(model, df)
        coherence = evaluation(topic_words, tokens, dictionary, corpus)
        coherences.append(coherence)
#         perplexities.append(perplexity)
        print("Finished evaluating model " + str(num))
        path = 'models/' + name + '_' + str(num) + '.pkl'
        pickle.dump(model, open(path, 'wb'))
        num += 1
    return coherences, models

In [ ]:
coherences, models = hyperparameter_tuning(df, 't2v_model')
t2v_model = models[np.argmax(coherences)]
# pickle.dump(model, open('models/t2v_model.pkl', 'wb'))

In [ ]:
# model = Top2Vec(df['preprocessed_hlead'].tolist(), umap_args={'n_neighbors': 10, 'n_components': 5, 'metric': 'cosine', 'low_memory': False, 'random_state': 42})

In [ ]:
print("Number of topics detected:", model.get_num_topics())

In [ ]:
topic_words, word_scores, topic_nums = model.get_topics()

for i in range(10):
    for j in range(10):
        str(word_scores[i][j])+'*'+str(topic_words[i][j])

In [ ]:
topic_words[0]

In [ ]:
topic_words, word_scores, topic_nums = model.get_topics()

for topic in topic_nums:
    model.generate_topic_wordcloud(topic)